# Dependencies

In [ ]:
%pip install pyautogui opencv-python easyocr --user

# Libraries

In [79]:
import cv2
import easyocr
import logging
import pyautogui
import numpy as np

In [80]:
def take_screenshot():
    screenshot = pyautogui.screenshot()
    screenshot = np.array(screenshot)
    
    return cv2.cvtColor(screenshot, cv2.COLOR_RGB2BGR)

def get_colors():
    color1 = np.array([81, 215, 170]) # Green light
    color2 = np.array([73, 209, 162]) # Green
    color3 = np.array([159, 194, 229]) # Nude light
    color4 = np.array([153, 184, 215]) # Nude
    color5 = np.array([58, 175, 135]) # line

    return color1, color2, color3, color4, color5

def create_mask_colors(screenshot, color1, color2, color3, color4, color5):
    mask1 = cv2.inRange(screenshot, color1, color1)
    mask2 = cv2.inRange(screenshot, color2, color2)
    mask3 = cv2.inRange(screenshot, color3, color3)
    mask4 = cv2.inRange(screenshot, color4, color4)
    mask5 = cv2.inRange(screenshot, color5, color5)

    return mask1, mask2, mask3, mask4, mask5

def get_table_contours(mask1, mask2, mask3, mask4, mask5):
    combined_mask = cv2.bitwise_or(mask1, mask2)
    combined_mask = cv2.bitwise_or(combined_mask, mask3)
    combined_mask = cv2.bitwise_or(combined_mask, mask4)
    combined_mask = cv2.bitwise_or(combined_mask, mask5)
    
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def get_table_dimensions(contours, screenshot):
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)

    table = screenshot[y:y+h, x:x+w]
    cv2.imwrite("./img/table.png", table)

    return x, y, w, h

In [81]:
screenshot = take_screenshot()
color1, color2, color3, color4, color5 = get_colors()
mask1, mask2, mask3, mask4, mask5 = create_mask_colors(screenshot, color1, color2, color3, color4, color5)

table_contours = get_table_contours(mask1, mask2, mask3, mask4, mask5)
table_dims = get_table_dimensions(table_contours, screenshot)

square_dims = {
    (450, 360): 45, # Easy
    (540, 420): 30, # Medium
    (600, 500): 25, # Hard
}

square_size = square_dims[(table_dims[2], table_dims[3])]

In [82]:
table_dims, square_size

((152, 197, 600, 500), 25)

In [106]:
def get_square(screenshot, table_dims, i, j, square_size):
    dx, dy = j * square_size, i * square_size
    x, y, w, h = (
        table_dims[0] + dx, 
        table_dims[1] + dy, 
        square_size, 
        square_size
    )
    return screenshot[y:y+h, x:x+w]

def scale_image(square, width, inter=cv2.INTER_AREA):
    original_height, original_width = square.shape[:2]
    ratio = width / float(original_width)
    height = int(original_height * ratio)

    return cv2.resize(square, (width, height), interpolation=inter)

def str_to_number(str_value):
    try:
        number = int(str_value)
        return number
    except ValueError:
        return None
    
def get_number(square):
    reader = easyocr.Reader(['en'])
    results = reader.readtext(square)

    array = np.array(results, dtype=object)
    sorted_indexes = np.argsort(array[:, 2])[::-1]

    sorted_array = array[sorted_indexes]
    number = int(sorted_array[0, 1])

    return number
    
def get_cell_value(square, colors):
    for index, color in enumerate(colors):
        mask = cv2.inRange(square, color, color)
        color_percentage = (np.sum(mask) / (mask.size * 255)) * 100
        
        if color_percentage > 90:
            return 0 if index < 2 else -1
    
    return get_number(square)

In [107]:
logging.getLogger().setLevel(logging.ERROR)

_, _, width, height = table_dims
n, m = height // square_size, width // square_size
table = np.zeros((n, m))

for i in range(n):
    for j in range(m):
        square = get_square(screenshot, table_dims, i, j, square_size)
        resized_square = scale_image(square, 5 * square_size)
        number = get_cell_value(resized_square, [color1, color2, color3, color4])
        table[i, j] = number
    print(f"Fila {i + 1} de {n} leida")

Fila 1 de 20 leida
Fila 2 de 20 leida
Fila 3 de 20 leida
Fila 4 de 20 leida
Fila 5 de 20 leida
Fila 6 de 20 leida
Fila 7 de 20 leida
Fila 8 de 20 leida
Fila 9 de 20 leida
Fila 10 de 20 leida
Fila 11 de 20 leida
Fila 12 de 20 leida
Fila 13 de 20 leida
Fila 14 de 20 leida
Fila 15 de 20 leida
Fila 16 de 20 leida
Fila 17 de 20 leida
Fila 18 de 20 leida
Fila 19 de 20 leida
Fila 20 de 20 leida


In [159]:
str_value = ""
for row in table:
    for col in row:
        value = f"  {int(col)}  " if col > -1 else f" {int(col)}  "
        str_value += value + ""
    str_value += "\n"

print(str_value)

  0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0  
  0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0  
  0    0    0    0    0    3    1    2    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0  
  0    0    0    0    0    2   -1    2    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0  
  0    0    2    1    1    1   -1    1    2    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0  
  0    0    1   -1   -1   -1   -1   -1    2    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0  
  0    0    2    1    1   -1   -1   -1    2    0    0    0    0    0    0    0    0    0    4    0    0    0    0    0  
  0    0    0    0    1   -1   -1   -1    1    1    0    0    0    0    0    0    0    0    0    0    0    0    0    0  
  0    0    0    0    1   -1   -

In [150]:
def get_cell(table, i, j):
    if 0 <= i < len(table) and 0 <= j < len(table[0]):
        return table[i][j]
    else:
        return None

def adjacent_info(table, i, j):
    empty_cells, mines = [], 0
    for dx, dy in [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]:
        ni, nj = i + dy, j + dx
        cell = get_cell(table, ni, nj)
        
        if cell == 0 and get_cell(table, i, j) > 0:
            empty_cells.append((ni, nj))
        mines += int(cell == -2)

    return empty_cells, mines
    
def get_num_empty_adjacent_cells(table, i, j):
    empty_cells, mines = set(), 0
    for dx, dy in [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]:
        ni, nj = i + dy, j + dx
        cell = get_cell(table, ni, nj)

        if cell is not None:
            emp_adj_cells, adj_mines = adjacent_info(table, ni, nj)
            empty_cells = empty_cells.union(set(emp_adj_cells))
            mines += cell - adj_mines

    return empty_cells, mines

def get_probability(table, i, j):
    if get_cell(table, i, j) == 0:
        empty_cells, mines = get_num_empty_adjacent_cells(table, i, j)
        return 1 if len(empty_cells) == 0 else (mines / len(empty_cells))
    
    return 1

In [152]:
prob_table = np.ones((n, m))
for i in range(n):
    for j in range(m):
        prob_table[i, j] = get_probability(table, i, j)